In [4]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import seaborn as sns
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

In [5]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2018-01-29", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True, treads = True)

[*********************100%***********************]  4 of 4 completed


In [6]:
df_comp = raw_data.copy()

In [7]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [8]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

# Creating Returns

In [9]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

# Splitting the Data

In [10]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

# Fitting a Model

In [12]:
from pmdarima.arima import auto_arima

In [13]:
model_auto = auto_arima(df.ret_ftse[1:])

In [14]:
model_auto

ARIMA(maxiter=50, method='lbfgs', order=(3, 0, 5), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=False)

In [15]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5019
Model:               SARIMAX(3, 0, 5)   Log Likelihood               -7883.727
Date:                Wed, 31 Mar 2021   AIC                          15785.453
Time:                        11:32:54   BIC                          15844.142
Sample:                             0   HQIC                         15806.019
                               - 5019                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.1273      0.078     -1.626      0.104      -0.281       0.026
ar.L2         -0.8760      0.020    -44.843      0.000      -0.914      -0.838
ar.L3         -0.2776      0.072     -3.879      0.000      -0.418      -0.137
ma.L1          0.1026      0.078      1.311      0.190      -0.051       0.256
ma.L2          0.8224      0.023     36.515      0.000       0.778       0.867
ma.L3          0.1648      0.070      2.362      0.018       0.028       0.302
ma.L4         -0.0085      0.009     -0.917      0.359      -0.027       0.010
ma.L5         -0.1114      0.008    -13.438      0.000      -0.128      -0.095
sigma2         1.3547      0.015     91.101      0.000       1.326       1.384
===================================================================================
Ljung-Box (Q):                       70.94   Jarque-Bera (JB):              6244.36
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.99   Skew:                            -0.17
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.45
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

# Important Arguments

In [16]:
model_auto = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample = int(len(df_comp)*0.2))


# !!! Important Note: In pdmarima v1.5.2, out_of_sample_size is replaced with out_of_sample, so make sure to use the latter!


# exogenous -> outside factors (e.g other time series)
# m -> seasonal cycle length
# max_order -> maximum amount of variables to be used in the regression (p + q)
# max_p -> maximum AR components
# max_q -> maximum MA components
# max_d -> maximum Integrations
# maxiter -> maximum iterations we're giving the model to converge the coefficients (becomes harder as the order increases)
# alpha -> level of significance, default is 5%, which we should be using most of the time
# n_jobs -> how many models to fit at a time (-1 indicates "as many as possible")
# trend -> "ct" usually
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' 
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple -> validates the model selection (pass the entire dataset, and set 20% to be the out_of_sample_size)

In [17]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                          SARIMAX Results                                           
====================================================================================================
Dep. Variable:                                            y   No. Observations:                 6275
Model:             SARIMAX(0, 0, 1)x(0, 0, [1, 2, 3, 4], 5)   Log Likelihood               -6333.669
Date:                                      Wed, 31 Mar 2021   AIC                          12689.339
Time:                                              11:56:07   BIC                          12763.526
Sample:                                          01-10-1994   HQIC                         12715.044
                                               - 01-26-2018                                         
Covariance Type:                                        opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0022      0.014     -0.160      0.873      -0.029       0.024
drift      -1.758e-06   4.19e-06     -0.420      0.675   -9.96e-06    6.45e-06
ret_spx        0.0958      0.006     17.043      0.000       0.085       0.107
ret_dax        0.5582      0.005    114.044      0.000       0.549       0.568
ret_nikkei     0.0702      0.004     16.696      0.000       0.062       0.078
ma.L1         -0.1062      0.007    -14.614      0.000      -0.120      -0.092
ma.S.L5       -0.0283      0.008     -3.361      0.001      -0.045      -0.012
ma.S.L10      -0.0497      0.009     -5.440      0.000      -0.068      -0.032
ma.S.L15      -0.0237      0.009     -2.720      0.007      -0.041      -0.007
ma.S.L20      -0.0242      0.009     -2.723      0.006      -0.042      -0.007
sigma2         0.4425      0.004    101.247      0.000       0.434       0.451
===================================================================================
Ljung-Box (Q):                       75.68   Jarque-Bera (JB):             14882.64
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.54   Skew:                             0.24
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.53
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""